In [35]:
import json
%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D
import pylab
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.mplot3d import proj3d


In [36]:
# read file
#with open('Electrum.json', 'r') as myfile:
with open('SkysenseAB.json', 'r') as myfile:
    data=myfile.read()

In [37]:
# parse file
obj = json.loads(data)

In [38]:
positions=[]
sizes=[]
colors=[]
for i in range(len(obj)):
    positions.append((obj[i]["Dimension"]["x_min"],obj[i]["Dimension"]["y_min"],0))
    sizes.append((obj[i]["Dimension"]["x_max"]-obj[i]["Dimension"]["x_min"],obj[i]["Dimension"]["y_max"]-obj[i]["Dimension"]["y_min"],
                 obj[i]["Dimension"]["z_max"]-obj[i]["Dimension"]["z_min"]))
    colors.append("gray")

In [39]:
# read file
with open('SkysenseAB1.json', 'r') as myfile:
#with open('Electrum copy.json', 'r') as myfile:
    data2=myfile.read()
# parse file
obj2 = json.loads(data2)

In [40]:
drone_x = []
drone_y = []
drone_z = []
drone_SNR = []
drone_thrgh = []
for i in range(len(obj2)):
    drone_x.append(obj2[i]["posX"])
    drone_y.append(obj2[i]["posY"])
    drone_z.append(obj2[i]["posZ"])
    drone_SNR.append(float(obj2[i]["SNR"]))
    drone_thrgh.append(float(obj2[i]["Throughput"]))

In [41]:
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

def cuboid_data(o, size=(1,1,1)):
    # code taken from
    # https://stackoverflow.comappenda/35978146/4124317
    # suppose axis direction: x: to left; y: to inside; z: to upper
    # get the length, width, and height
    l, w, h = size
    x = [[o[0], o[0] + l, o[0] + l, o[0], o[0]],
         [o[0], o[0] + l, o[0] + l, o[0], o[0]],
         [o[0], o[0] + l, o[0] + l, o[0], o[0]],
         [o[0], o[0] + l, o[0] + l, o[0], o[0]]]
    y = [[o[1], o[1], o[1] + w, o[1] + w, o[1]],
         [o[1], o[1], o[1] + w, o[1] + w, o[1]],
         [o[1], o[1], o[1], o[1], o[1]],
         [o[1] + w, o[1] + w, o[1] + w, o[1] + w, o[1] + w]]
    z = [[o[2], o[2], o[2], o[2], o[2]],
         [o[2] + h, o[2] + h, o[2] + h, o[2] + h, o[2] + h],
         [o[2], o[2], o[2] + h, o[2] + h, o[2]],
         [o[2], o[2], o[2] + h, o[2] + h, o[2]]]
    return np.array(x), np.array(y), np.array(z)

def plotCubeAt(pos=(0,0,0), size=(1,1,1), ax=None,**kwargs):
    # Plotting a cube element at position pos
    if ax !=None:
        X, Y, Z = cuboid_data( pos, size )
        ax.plot_surface(X, Y, Z, rstride=1, cstride=1, **kwargs)

#positions = [(20,-20,0),(-20,-20,0)]
#sizes = [(10,40,40), (10,40,40)]
#colors = ["gray","gray"]

#drone path
#drone_x = [50, 0, -50, 0]
#drone_y = [0, 50, 0, -50]
#drone_z = [50, 50, 50, 50]

#drone_SNR = [30, 25, 10, 20]
#drone_thrgh = [64, 34, 39, 50]


fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter(0, 0, 9, c='red') #AP (0,0,0)
#ax.scatter(50,50,100, c='b') #drone (x,y,z)
#ax.text(50,50,100,"snr:78", fontsize=12, color = "b")
#ax.scatter(-50,-50,100, c='b') #drone (x,y,z)
#ax.text(-50,-50,100,"snr:20", fontsize=12, color = "b")
#ax.scatter(50,50,50, c='b') #drone (x,y,z)
#ax.text(50,50,50,"snr:36", fontsize=12, color = "b")
for p,s,c in zip(positions,sizes,colors):
    plotCubeAt(pos=p, size=s, ax=ax, color=c)

ax.set_xlim([-160,160])
ax.set_ylim([-200,250])
ax.set_zlim([0,250])
# Writing The Title of The Plot
ax.set_title(r'$Graphical\; Representation\; $', fontsize=10)

# Stablishing the plots of our legend labels
gray_proxy = plt.Rectangle((0, 0), 1, 1, fc='gray')
red_proxy = plt.Line2D([0], [0], linestyle="none", marker='o', markersize=10, color='red')
blue_proxy = plt.Line2D([0], [0], linestyle="none", marker='o', markersize=10, color='blue')

# Drawing Our Legend
ax.legend([gray_proxy, red_proxy, blue_proxy], [r'$Buildings$', r'$AP\; (0,0,0)$', r'$Drone\; path$'], numpoints=1, loc='upper left')

for k in range(len(drone_x)):
    scatter_node = ax.scatter(drone_x[k], drone_y[k], drone_z[k], c='b')  # drone (x,y,z)

    x2, y2, _ = proj3d.proj_transform(drone_x[k],drone_y[k],drone_z[k], ax.get_proj())

    label = pylab.annotate(
        r'$SNR=%.2f$ dB' '\n' '$thrgh=%.2f $ Mbps' % (drone_SNR[k],drone_thrgh[k],),
        xy = (x2, y2), xytext = (-20, 20),
        textcoords = 'offset points', ha = 'right', va = 'bottom',
        bbox = dict(boxstyle = 'round,pad=0.5', fc = 'wheat', alpha = 0.2),
        arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))

    plt.pause(2)
    scatter_node.set_visible(False)
    label.remove()

pylab.show()


plt.show()